In [ ]:
import pandas as pd
training = pd.read_csv('trainingVect.csv')


In [ ]:
training['Ow2'] = 0
training.ix[training['word2'].str[-1] == 'O','Ow2'] = 1

In [ ]:
heldout = pd.read_csv('heldoutVect.csv')

In [ ]:
dvan = heldout[['word1','word2','tag','label']][heldout.word2.str[-1]== 'O'].groupby('label').count()
dvan.sort_values(by='word1',ascending=False)

In [ ]:
heldout[['word1','word2','tag','label']][(heldout['word2'].str[-2:] == 'EH') & (heldout['label']==2)]

In [ ]:
heldout['Ow2'] = 0
heldout.ix[training['word2'].str[-1] == 'O','Ow2'] = 1

In [ ]:
heldout.head(4)

## Ignore

In [ ]:
finRemove = eval(open('adap2.features').read())
type(finRemove)
finRemove[0]

In [ ]:
for item in finRemove:
    del heldout[item]
    del training[item]

In [ ]:
heldout.columns.difference(training.columns)

In [ ]:
print len(heldout.columns),len(training.columns)

In [ ]:
for i,item in enumerate(training.columns):
    if training.columns[i] != heldout.columns[i]:
        print training.columns[i],heldout.columns[i]

## Stop Ignorring

In [ ]:
print training.columns[-41:]

allCols = list(training.columns[5:-39])
allCols.insert(-1,'Ow2')
allCols[-3:]

In [ ]:
from numpy import *


#featVect = featVect.sample(frac=1.0)
features = array(training.ix[:,allCols])
labels = array(training.ix[:,'label'])

heldFeatures = array(heldout.ix[:,allCols])

heldLabels = array(heldout.ix[:,'label'])
print allCols[-2:]

In [ ]:
a = training.columns
a[26:413]


## Random Forests

In [ ]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(labels, n_folds=10,shuffle=True)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score



In [ ]:
clf = RandomForestClassifier(n_jobs=-1,n_estimators=400,max_features='log2')
clf  = clf.fit(features,labels)
predicted_label = clf.predict(heldFeatures)
pred_prob = clf.predict_proba(heldFeatures)

In [ ]:
print classification_report(heldLabels, predicted_label)
print ('Accuracy:',accuracy_score(heldLabels, predicted_label))

In [ ]:
print classification_report(heldLabels, predicted_label)
print ('Accuracy:',accuracy_score(heldLabels, predicted_label))

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
forest = clf
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
X = features
# Print the feature ranking
print("Feature ranking:")

"""for f in range(features.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
"""
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
triple = zip(range(len(heldLabels)),heldLabels,predicted_label)


In [ ]:
prob =pd.DataFrame().from_records(pred_prob,columns=['prob0','prob1','prob2','prob3'])
prob = pd.merge(prob,result,left_index=True,right_index=True,how='inner')
prob[(prob['heldLabels']!= prob['predicted_label']) & (prob['heldLabels'] == 2)][['prob0','prob1','prob2','prob3','word1','word2','heldLabels','predicted_label','dups','tag']]

In [ ]:
prob[(prob['word1'].str[-1] == 'a') & (prob['word2'].str[-1] == 'm') ]

In [ ]:
trip = pd.DataFrame().from_records(triple,columns=['index','heldLabels','predicted_label'])

result = pd.merge(trip,heldout,how='inner',left_on='index',right_index=True,)
result[(result['heldLabels']!= result['predicted_label']) & (result['heldLabels'] == 2)][['word1','word2','heldLabels','predicted_label','dups','tag']]

## Extremely Randomied trees

In [ ]:
from sklearn.ensemble import RandomTreesEmbedding, ExtraTreesClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
trees = ExtraTreesClassifier(n_estimators=400, random_state=43, max_features='log2')
trees = trees.fit(features,labels)

predicted_label = trees.predict(heldFeatures)
print classification_report(heldLabels, predicted_label)
print ('Accuracy:',accuracy_score(heldLabels, predicted_label))



In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(features, labels)  
predicted_label = clf.predict(heldFeatures)
print classification_report(heldLabels, predicted_label)
print ('Accuracy:',accuracy_score(heldLabels, predicted_label))

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(features, labels)  
predicted_label = clf.predict(heldFeatures)
print classification_report(heldLabels, predicted_label)
print ('Accuracy:',accuracy_score(heldLabels, predicted_label))

In [ ]:
import numpy
alist = zip(list(a[5:-29]),list(trees.feature_importances_))
feati = open('featImp.csv','w')
for item in alist:
    print >>feati,item[0],',',item[1]
feati.close()

## GBM

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=10).fit(features,labels)
predicted_label = clf.predict(heldFeatures)


In [ ]:
print classification_report(heldLabels, predicted_label)
print ('Accuracy:',accuracy_score(heldLabels, predicted_label))

## Easy Ensemble

In [ ]:
from collections import Counter
from imblearn.ensemble import BalanceCascade
from imblearn.ensemble import EasyEnsemble
ee = EasyEnsemble(ratio=0.055,random_state=25,replacement=True,n_subsets=20)
X_res, y_res = ee.fit_sample(features, labels)
print('Resampled dataset shape {}'.format(Counter(y_res[1])))

"""from sklearn.decomposition import PCA
pcaHeld = PCA(n_components=900).fit(features)
pcaFe = pcaHeld.transform(features)"""

In [ ]:
X_res[1].shape

In [ ]:
## from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

#clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=100,splitter='best'),n_estimators=100,learning_rate=1.0)
clf = ExtraTreesClassifier(n_estimators=200, max_features='log2', random_state=43, warm_start=True,n_jobs=-1)
countrs= []
for i in range(len(heldFeatures)):
    countrs.append([0]*4)

for i,item in enumerate(X_res):
    print i,
    clf.fit(X_res[i],y_res[i])
    staged_predict = clf.predict_proba(heldFeatures)
    clf.n_estimators += 50
    for j,stuff in enumerate(staged_predict):
        for k,thing in enumerate(list(stuff)):
            countrs[j][k] += thing

In [ ]:
majLabels = [-1]*len(countrs)
for i,item in enumerate(countrs):
    majLabels[i] = item.index(max(item))
print classification_report(heldLabels, majLabels)
print ('Accuracy:',accuracy_score(heldLabels, majLabels))

In [ ]:
preds = open('preds.csv','w')
for i,item in enumerate(heldLabels):
    print>>preds,item,',',majLabels[i]
preds.close()


In [ ]:
print type(heldLabels)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import pandas 
preds = pandas.read_csv('myList.csv')
import numpy
a = list(preds.label)
b = list(preds.predicted)
print classification_report(a,b)
print ('Accuracy:',accuracy_score(a, b))

In [ ]:
preds[(preds['tag']=='Bsmn')].groupby('predicted').count()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import pandas 
preds = pandas.read_csv('preds2.csv')
import numpy
a = list(preds.orig)
b = list(preds.pred)
print classification_report(a,b)
print ('Accuracy:',accuracy_score(a, b))

In [ ]:
clfFull = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=100,splitter='best'),n_estimators=600,learning_rate=1.0)
clfFull = clfFull.fit(features,labels)
predicted_label = clfFull.predict(heldFeatures)
print classification_report(heldLabels, predicted_label)
print ('Accuracy:',accuracy_score(heldLabels, predicted_label))

In [ ]:
import numpy as np
akie = training.select_dtypes(include=['float64']).columns
akie = akie.insert(-1,'label')
pkie = training[akie].groupby('label').sum().T
df = pkie
df['sum'] = df.sum(axis=1)
df['entropy'] = (df.ix[:,:3].div(df['sum'],axis=0)*np.log(df.ix[:,:3].div(df['sum'],axis=0))).sum(axis=1)
pkie.to_csv('summar1.csv')

In [ ]:
training[training['label']==2][['word1','word2','tag','label','dups']]

In [ ]:
ap = list(training[training['tag'] == 'Ds' ]['word2'])
ap1 = list(training[training['tag'] == 'Ds' ]['word1'])
#vaswra	2.6.115.2.1	napuM.	manuRyavargaH	vaswram										vaswram

In [ ]:
kAndas = pd.read_csv('all_kANdas.csv')
jusWord = list(kAndas['%0_Word'])


In [ ]:
trainFi = [item.split(',') for item in open('trainigFiltered.csv').read().splitlines()[1:]]
import pandas as pd
merger = pd.merge(training,kAndas,how='left',left_on='word1',right_on='%0_Word',suffixes=('_x','_word1'))
merger = pd.merge(merger,kAndas,how='left',left_on='word2',right_on='%0_Word',suffixes=('_x','_word2'))
merger
cat_columns =merger.select_dtypes(include=['object']).columns

cat_columns = cat_columns[7:]
for item in cat_columns:
    merger[item] = merger[item].astype('category')

merger

In [ ]:
ctr = 0
for i,item in enumerate(ap):
    if item not in jusWord:
        if item[:-1] not in jusWord:
            if item[:-3] not in jusWord:
                if item[:-1]+'a' not in jusWord:
                    print i,item
                    ctr += 1
    
print ctr,len(ap)

#word1 = -ya+a, -aXi+, -pra+, -+n
#word2 = -m+ -ena+a -e+a
## vit, AkarRaNa, saMmAna


In [ ]:
f = [item.split(',') for item in open('heldoutFiltered.csv').read().splitlines()[1:]]
kAndas = [item.split(',') for item in open('all_kANdas.csv').read().splitlines()]
nanList = ['None']*(len(kAndas[-1]) - 3)
ctr = 0
ctr2 = 0
for i,item in enumerate(f):
    matched = 0
    matched2 = 0
    for j,stuff in enumerate(kAndas):
        if item[1] == stuff[0]:
            f[i].extend(kAndas[j][2:])
            matched= 1
            ctr += 1
            break
    matched2 = 0
    if matched == 0:
        f[i].extend(nanList)

    for j,stuff in enumerate(kAndas):
        if item[2] == stuff[0]:          
            f[i].extend(kAndas[j][2:])
            matched2= 1
            ctr2 += 1
            break
        elif item[2][:-1] == stuff[0] or item[2][:-3] == stuff[0] or item[2][:-1]+'a' == stuff[0] or item[2][:-2]+'a' == stuff[0]:
            f[i].extend(kAndas[j][2:])
            matched2= 1
            ctr2 += 1
            break
            
    if matched2 == 0:
        f[i].extend(nanList)

In [ ]:
import pandas as pd
training = pd.read_csv('trainigFiltered.csv')
training[(training['tag'] == 'Ds') | (training['tag'] == 'Di')] [['word1','word2']].to_csv('dvandvas.csv',index=False)
f = open('dvandvas.csv').read().replace(',','$').splitlines()
k = open('dvandvas.dat','w')
for item in f:
    for chara in item:
        print>>k,chara,
    print >> k,'\n'
k.close()

In [ ]:
newTrainingFiltered = open('newHeldoutFiltered.csv','w')
for item in f:
    print >> newTrainingFiltered,'\n',
    for stuff in item:
        print >> newTrainingFiltered,stuff,',',
newTrainingFiltered.close()

In [ ]:
filna = 'newHeldoutFiltered.csv'
reindex = pd.read_csv(filna).sample(frac=1)
del reindex['index']
reindex.to_csv('newTrainingFiltered.csv')